# Carregar, limpar e organizar um conjunto de dados usando o pacote pandas

Códigos apresentados no vídeo [Limpeza de dados em python: pandas no google colab](https://https://www.youtube.com/watch?v=0SgghZ5S87o)

## Dados de seguro rural

Link: https://dados.agricultura.gov.br/dataset/sisser3

No site há dados disponíveis de 2006 a 2022.

Importar pacotes necessários:

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive, files
import os

pd.options.display.float_format = '{:.2f}'.format    # pandas: para todos os números aparecerem com duas casas decimais

In [ ]:
import sys
print(sys.version)
print(pd.__version__)

É possível baixar os dados com o link ou baixar o conjunto de dados para o computador e salvar no google drive.

In [ ]:
# download diretamente do site
# !wget https://dados.agricultura.gov.br/dataset/baefdc68-9bad-4204-83e8-f2888b79ab48/resource/e6f95018-6c19-426a-9a62-fc9e5bfc721b/download/psrdadosabertos2016a2021excel.xlsx
# dados = pd.read_excel("psrdadosabertos2016a2021excel.xlsx", decimal=',')

Eu já salvei os dados no meu *google drive* e vou acessar a partir dele:

In [ ]:
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/')
dados = pd.read_excel("psrdadosabertos2016a2021excel.xlsx", decimal=',')

In [ ]:
# número de linhas e colunas
dados.shape

Cada uma das 698.675 linhas é uma apólice contratada por um produtor.

Vamos selecionar uma parte menor dos dados (apenas dados de 2021) para que possamos trabalhar com um conjunto menor:

In [ ]:
dados21 = dados[dados['ANO_APOLICE'] == 2021]
dados21.shape

## Visão geral dos dados


Neste notebook vamos trabalhar apenas com os dados de 2021. Eles estão salvos no objeto 'dados21'. Vou fazer uma  cópia deles num objeto chamado 'seg':

In [ ]:
seg = dados21.copy()

In [ ]:
# número de linhas e colunas (observações e variáveis)
seg.shape

In [ ]:
# para visualizar até 50 colunas ou o número desejado
pd.set_option('display.max_columns', 50)

In [ ]:
seg.head(10)

In [ ]:
seg.tail()

In [ ]:
seg.sample(10)

Se quisermos iniciar os índices das linhas a partir do zero:

In [ ]:
seg = seg.reset_index(drop=True)

In [ ]:
seg.head()

In [ ]:
seg.columns

In [ ]:
seg.info()

## Filtrando linhas

Não salvaremos os dados filtrados. Faremos apenas algumas consultas.

In [ ]:
# no .iloc usamos os índices
seg.iloc[:10,:5]

In [ ]:
# no .loc usamos os nomes
seg.loc[:, ['NR_AREA_TOTAL', 'NR_PRODUTIVIDADE_ESTIMADA', 'VL_LIMITE_GARANTIA',
            'VL_PREMIO_LIQUIDO', 'VL_SUBVENCAO_FEDERAL', 'NR_APOLICE']]

In [ ]:
seg.loc[seg['NM_MUNICIPIO_PROPRIEDADE'] == 'Varginha']

In [ ]:
seg.loc[(seg['NM_MUNICIPIO_PROPRIEDADE'] == 'Varginha') & (seg['NM_CULTURA_GLOBAL'] == 'Café')]

In [ ]:
# o mesmo com query
seg.query('(NM_MUNICIPIO_PROPRIEDADE == "Varginha") and (NM_CULTURA_GLOBAL == "Café")')

## Resumindo os dados

In [ ]:
seg.describe()

In [ ]:
# quantis
seg['NR_AREA_TOTAL'].quantile([0.25, 0.5, 0.75])

In [ ]:
# algumas medidas estatísticas para algumas variáveis
seg[['VL_LIMITE_GARANTIA', 'VL_PREMIO_LIQUIDO', 'VL_SUBVENCAO_FEDERAL']].agg(['mean', 'min', 'max'])

In [ ]:
# mostrar as ocorrências de determinada variável
seg['NM_RAZAO_SOCIAL'].unique()

In [ ]:
# mostrar o número de ocorrências de cada categoria de uma variável
# normalize=True faz o cálculo da porcentagem
seg['NM_RAZAO_SOCIAL'].value_counts(normalize=True)

## Limpeza e organização dos dados

Lidar com dados faltantes:

In [ ]:
# sem sum() temos apenas True e False
seg.isna().sum()

Substituir faltantes por algum valor (se for necessário):

In [ ]:
seg = seg.fillna(0)

Substituir faltantes de uma variável pela média dessa variável (se for necessário):

In [ ]:
seg['VL_PREMIO_LIQUIDO'] = seg['VL_PREMIO_LIQUIDO'].fillna(seg['VL_PREMIO_LIQUIDO'].mean())

Para realizar a limpeza e organização dos dados, vamos criar um novo objeto chamado 'df' para que os dados originais continuem no objeto 'seg':

In [ ]:
df = seg.replace(['-', '...', 'X'], 0)

In [ ]:
# se todas as linhas forem NaN, apagar a linha
df.dropna(inplace=True, how='all')

In [ ]:
# se qualquer linha for NaN, apagar a linha
df.dropna(inplace=True)

In [ ]:
df.shape

In [ ]:
df.columns

Criação de nova coluna: vamos calcular o tempo de vigência (em dias) de cada apólice e colocar essa informação em uma nova coluna chamada 'duracao'.

In [ ]:
df['DT_FIM_VIGENCIA'] = pd.to_datetime(df['DT_FIM_VIGENCIA'], errors = 'coerce')
df['duracao'] = (df['DT_FIM_VIGENCIA'] - df['DT_INICIO_VIGENCIA']).dt.days

In [ ]:
# ordenar os valores dessa nova coluna
df.sort_values('duracao', ascending=False)

In [ ]:
# checar se algum NaN apareceu
df.isna().sum()

In [ ]:
# verificar qual é o NaN que apareceu
df[df.isna().any(axis=1)]

In [ ]:
# retirar a linha com NaN
df.dropna(inplace=True)

Algumas colunas não serão usadas, vamos retirá-las com a função *drop*:

In [ ]:
retirar = ['CD_PROCESSO_SUSEP', 'NR_PROPOSTA', 'ID_PROPOSTA',
       'DT_PROPOSTA', 'DT_INICIO_VIGENCIA', 'DT_FIM_VIGENCIA', 'NM_SEGURADO',
       'NR_DOCUMENTO_SEGURADO','LATITUDE', 'NR_GRAU_LAT', 'NR_MIN_LAT',
       'NR_SEG_LAT', 'LONGITUDE', 'NR_GRAU_LONG', 'NR_MIN_LONG', 'NR_SEG_LONG',
       'NR_DECIMAL_LATITUDE', 'NR_DECIMAL_LONGITUDE', 'DT_APOLICE', 'ANO_APOLICE']
df = df.drop(retirar, axis=1)

In [ ]:
df.columns

A variável 'EVENTO_PREPONDERANTE' possui alguns caracteres descnecessários ('\xa0'). Vamos corrigir isso:

In [ ]:
df['EVENTO_PREPONDERANTE'].unique()

In [ ]:
df['EVENTO_PREPONDERANTE'] = df['EVENTO_PREPONDERANTE'].str.split().str.join(' ')

In [ ]:
df['EVENTO_PREPONDERANTE'] = df['EVENTO_PREPONDERANTE'].replace([np.NaN], '0')

In [ ]:
df['EVENTO_PREPONDERANTE'].unique()

In [ ]:
df.shape

Retirar espaços em branco do início e do final dos nomes (caso haja):

In [ ]:
df['NM_RAZAO_SOCIAL'] = df['NM_RAZAO_SOCIAL'].str.strip()
df['NM_CULTURA_GLOBAL'] = df['NM_CULTURA_GLOBAL'].str.strip()

Uso do *groupby*:

In [ ]:
df.groupby('SG_UF_PROPRIEDADE')['VL_PREMIO_LIQUIDO'].sum()

Calcular o índice de sinistralidade:

O índice de sinistralidade mede (em %) a participação do valor
total das indenizações pagas aos produtores rurais em decorrência de sinistros em relação ao valor do prêmio total arrecadado pelas seguradoras. O valor limite de 65% é tido como um índice de sinistralidade aceitável. O índice de sinistralidade permite que se façam inferências sobre o equilíbrio atuarial das operações
de seguro rural.

In [ ]:
df['VALOR_INDENIZAÇÃO'].sum() / df['VL_PREMIO_LIQUIDO'].sum() * 100

In [ ]:
df.columns

Obs.: Valores monetários estão expressos em R$mil

In [ ]:
df.columns = ['seguradora', 'nome_mun', 'uf',
              'tipo', 'cultura', 'area', 'animal',
              'prod_est', 'prod_seg', 'nivel_cob',
              'total_seg', 'premio', 'taxa',
              'subvencao', 'apolice', 'mun', 'indenizacao',
              'evento', 'duracao']

In [ ]:
df.info()

In [ ]:
# transformar o código do município para inteiro
df['mun'] = df['mun'].astype(int)

Criar coluna de sinistralidade média:

In [ ]:
df['sinistralidade'] = df['indenizacao'] / df['premio']

In [ ]:
df.sample(10)

Criar nova coluna chamada sinistro, cujo valor será 0 se a coluna 'evento' for 0, ou seja, não houve sinistro, ou será 1 se tiver algum conteúdo diferente de 0.

In [ ]:
df['sinistro'] = np.where(df['evento'].str.isnumeric(), 0, 1)

In [ ]:
df.describe()

Organizar as variáveis usando .loc:

In [ ]:
df = df.loc[:, ['apolice', 'mun', 'nome_mun', 'uf', 'seguradora', 'tipo',
                'cultura', 'area', 'animal', 'duracao',
                'prod_est', 'prod_seg', 'nivel_cob', 'total_seg',
                'premio', 'taxa', 'subvencao', 'indenizacao',
                'evento', 'sinistro', 'sinistralidade']]

In [ ]:
df.info()

Ordenar os dados:

In [ ]:
df.sort_values('premio')

In [ ]:
df.columns

In [ ]:
df[['nome_mun', 'uf', 'subvencao']].sort_values('subvencao', ascending=False)

## Salvar os dados

Podemos salvar os dados de 2021 com as alterações que fizemos.

In [ ]:
# checar o dataframe
df.shape

Se eu li os dados a partir do *google drive*, o arquivo será salvo lá. Senão, ele aparecerá no próprio ambiente colab (à esquerda).

In [ ]:
# salvar o dataframe como xlsx
df.to_excel("seguro-rural-2021.xlsx", index=False)

In [ ]:
# se quiser salvar o dataframe como csv
# df.to_csv("seguro-rural-2021.csv", index=False, encoding='latin1')

## Verificar se os dados serão lidos corretamente:

In [ ]:
# demora um pouco
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/')
dados = pd.read_excel("seguro-rural-2021.xlsx")

In [ ]:
dados.shape

In [ ]:
dados.sample(10)